Rules Perancangan Sistem Pakar Dengan Metode Forward Chaining dan
Certainty Factor Untuk Mendeteksi Penyakit Kelinci

In [35]:
from google.colab import drive

import matplotlib.pyplot as plt

In [36]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [37]:
# --- TAHAP 1 : MENGHUBUNGKAN GOOGLE DRIVE ---
# Jalankan sel ini untuk menghubungkan Colab dengan Google Drive Anda.

# The drive has already been mounted in the cell below to '/content/drive'.
# You can access files in your Drive through this path (e.g., '/content/drive/MyDrive/sispak/rules.json')

# from google.colab import drive
# drive.mount('/drive/MyDrive/sispak/rules.json')

# print("✅ Google Drive berhasil terhubung!")

# Note: This cell is no longer needed as the drive is mounted in cell Fd5X92Ipq_uo

In [38]:
# --- TAHAP 2 : MENGIMPOR LIBRARY JSON ---
# Mengimpor library 'json' untuk membaca file rules.json
import json

def load_rules(filepath):
    """
    Fungsi ini memenuhi instruksi "Membaca rules.json".
    Fungsi ini membuka file JSON, mem-parsing isinya,
    dan mengembalikannya sebagai list dictionary Python.
    """
    # Membuka file dalam mode 'r' (read/baca)
    with open(filepath, 'r') as f:
        # json.load() membaca file dan mengembalikan datanya
        return json.load(f)

def combine_cf(cf1, cf2):
    """
    Fungsi ini memenuhi instruksi "Menghitung nilai CF (menggabungkan rule)".
    Fungsi ini mengimplementasikan rumus CFcombine dari Jurnal (file 1).
    """

    # Rumus 1: Jika keduanya positif (dari Jurnal)
    # CFcombine = CF1 + CF2 * (1 - CF1)
    if cf1 > 0 and cf2 > 0:
        return cf1 + cf2 * (1 - cf1)

    # Rumus 3: Jika keduanya negatif (dari Jurnal)
    # CFcombine = CF1 + CF2 * (1 + CF1)
    elif cf1 < 0 and cf2 < 0:
        return cf1 + cf2 * (1 + cf1)

    # Rumus 2: Jika salah satu negatif/campuran (dari Jurnal)
    # CFcombine = (CF1 + CF2) / (1 - min(|CF1|, |CF2|))
    else:
        # Menghindari pembagian dengan nol jika min(abs(cf)) == 1
        denominator = 1 - min(abs(cf1), abs(cf2))
        if denominator == 0:
            return 1.0 if (cf1 + cf2) > 0 else -1.0
        return (cf1 + cf2) / denominator

print("✅ Fungsi load_rules() dan combine_cf() berhasil didefinisikan.")

✅ Fungsi load_rules() dan combine_cf() berhasil didefinisikan.


In [39]:
# --- TAHAP 3 : MEMBUAT ALGORITMA MESIN INFERENSI ---
# Fungsi Inti Mesin Inferensi (Forward Chaining)

def run_inference_engine(initial_facts, rules):
    """
    Fungsi ini adalah inti dari 'inference_engine'.
    Ini mengimplementasikan algoritma FORWARD CHAINING dengan benar.

    :param initial_facts: Dict fakta awal dari user (cth: {'G01': 0.6})
    :param rules: List aturan dari load_rules()
    :return: Dict fakta final (termasuk penyakit & komplikasi)
    """

    # 'facts' adalah "working memory" (memori kerja) kita.
    # Dimulai dengan fakta yang diberikan pengguna (gejala).
    facts = initial_facts.copy()

    # 'facts_newly_derived' adalah flag untuk mengontrol loop Forward Chaining.
    facts_newly_derived = True

    # --- FORWARD CHAINING LOOP ---
    # Loop akan terus berjalan selama kita masih menemukan fakta baru.
    while facts_newly_derived:
        facts_newly_derived = False

        # 'derived_cf_values' HARUS di-reset di dalam loop
        # agar kita bisa mengevaluasi ulang aturan di setiap iterasi.
        derived_cf_values = {}

        # Iterasi melalui SETIAP aturan dalam 'rules'
        for rule in rules:
            rule_id = rule['id']
            premises = rule['if']      # Daftar premis (cth: ["G01"])
            conclusion = rule['then']  # Kesimpulan (cth: "P01")
            cf_pakar = rule['cf_pakar'] # Nilai CF dari pakar

            # --- 1. CEK PREMIS (Kecocokan Aturan) ---
            all_premises_met = True
            min_cf_premis = 1.0

            for premis in premises:
                if premis not in facts:
                    all_premises_met = False
                    break
                else:
                    min_cf_premis = min(min_cf_premis, facts[premis])

            # --- 2. EKSEKUSI ATURAN (Aturan "Fire") ---
            if all_premises_met:

                cf_gejala = 0.0

                # Cek apakah premisnya adalah fakta awal (gejala)
                if premises[0] in initial_facts:
                    # KASUS 1: Aturan GEJALA -> PENYAKIT (cth: IF G01 THEN P01)
                    cf_user = facts[premises[0]]
                    cf_gejala = cf_user * cf_pakar
                else:
                    # KASUS 2: Aturan SEKUENSIAL (cth: IF P01 THEN K01)
                    cf_gejala = min_cf_premis * cf_pakar

                # --- 3. SIMPAN HASIL CF (untuk Aturan Paralel) ---
                if conclusion not in derived_cf_values:
                    derived_cf_values[conclusion] = []



                # Kita HARUS menambahkan semua CF, meskipun nilainya sama,
                # karena berasal dari aturan yang berbeda.
                derived_cf_values[conclusion].append(cf_gejala)


        # --- 4. GABUNGKAN (COMBINE) ATURAN PARALEL  ---
        # Logika ini sekarang ada DI DALAM 'while' loop.

        for conclusion, cf_list in derived_cf_values.items():
            if not cf_list:
                continue

            # Urutkan daftar untuk konsistensi perhitungan
            cf_list.sort(reverse=True)

            combined_cf = cf_list[0]
            if len(cf_list) > 1:
                for i in range(1, len(cf_list)):
                    combined_cf = combine_cf(combined_cf, cf_list[i])

            # --- 5. PERBARUI FAKTA ---
            # Cek apakah ini fakta baru ATAU fakta yang lebih pasti
            if conclusion not in facts or combined_cf > facts[conclusion]:
                # Perbarui 'facts' utama
                facts[conclusion] = combined_cf

                # Beri tahu loop 'while' bahwa kita
                # menemukan fakta baru, sehingga loop harus berjalan lagi
                # untuk memeriksa aturan sekuensial.
                facts_newly_derived = True

    # --- 6. KEMBALIKAN HASIL ---
    # Setelah loop 'while' selesai (tidak ada fakta baru yang ditemukan),
    # kembalikan 'facts' yang sudah lengkap.
    return facts

In [40]:
  # --- TAHAP 4 : MENGHUBUNGKAN KE UI ---
  # Mengimpor library ipywidgets untuk membuat UI interaktif di Colab
import ipywidgets as widgets
# Mengimpor display (untuk menampilkan widget) dan clear_output (untuk membersihkan hasil)
from IPython.display import display, clear_output

# --- 1. Definisi Data (dari Jurnal) ---

# [cite_start]Mendefinisikan dictionary untuk nama Gejala (dari Tabel 2 [cite: 233])
daftar_gejala = {
    "G01": "Nafsu makan hilang", "G02": "Berat badan turun", "G03": "Kulit kemerah-merahan",
    "G04": "Badan penuh koreng", "G05": "Bulu rontok", "G06": "Gatal-gatal pada tubuh",
    "G07": "Sulit hamil/keguguran", "G08": "Kulit telinga kemerah-merahan",
    "G09": "Area sekitar mata berwarna merah", "G10": "Badan Lemas", "G11": "Diare",
    "G12": "Muntah", "G13": "Suhu tubuh tinggi", "G14": "Menggaruk daun telinga",
    "G15": "Keluar cairan dalam lubang telinga", "G16": "Gatal dan sakit pada telinga",
    "G17": "Keluar cairan bernanah dari mata", "G18": "Radang kelopak mata berwarna merah",
    "G19": "Tulang kaki lemah dan bengkok", "G20": "Suka makan bulu", "G21": "Kanibal",
    "G22": "Tidak kuat menahan berat badan", "G23": "Keluar cairan dalam hidung",
    "G24": "Sesak nafas"
}

# [cite_start]Mendefinisikan dictionary untuk nama Penyakit (dari Tabel 1 [cite: 226] + Modifikasi kita)
daftar_penyakit = {
    "P01": "Scabies", "P02": "Gastroenteritis", "P03": "Radang Telinga",
    "P04": "Radang Mata", "P05": "Hipocalcium", "P06": "Pneumonia",
    "K01_Infeksi_Sekunder": "Komplikasi: Infeksi Kulit Sekunder (MODIFIKASI)",
    "K02_Gagal_Napas_Akut": "Komplikasi: Gagal Napas Akut (MODIFIKASI)"
}

# [cite_start]Mendefinisikan list of tuples untuk opsi dropdown keyakinan user (dari Tabel 3 [cite: 238])
# Format: (Teks yang Tampil, Nilai CF)
cf_user_options = [
    ("Pilih Keyakinan", 0.0), # Default, 0.0 agar diabaikan
    ("Pasti Tidak", -1.0),
    ("Hampir Pasti Tidak", -0.8),
    ("Kemungkinan Besar Tidak", -0.6),
    ("Mungkin Tidak", -0.4),
    ("Tidak Tahu", 0.0), # Di jurnal [-0.2, 0.2], kita pakai 0.0 agar mudah
    ("Mungkin Ya", 0.4),
    ("Kemungkinan Besar Ya", 0.6),
    ("Hampir Pasti Ya", 0.8),
    ("Pasti Ya", 1.0)
]

# --- 2. Buat Instance Widget UI ---
# Membuat 24 set Checkbox dan Dropdown

print("--- 🩺 Sistem Pakar Deteksi Penyakit Kelinci ---")
print("Silakan pilih gejala yang dialami kelinci dan tingkat keyakinan Anda:")

# Menambah ui_widgets
ui_widgets = {}
ui_elements_to_display = [] # List untuk menampung widget yang akan ditampilkan

# Looping melalui setiap gejala di 'daftar_gejala'
for kode, nama in daftar_gejala.items():

    checkbox = widgets.Checkbox(value=False, description=nama, layout={'width': '400px'})

    dropdown = widgets.Dropdown(options=cf_user_options, value=0.0, layout={'width': '200px'})

    # Simpan pasangan (checkbox, dropdown) ke dictionary 'ui_widgets'
    # Kita TETAP menggunakan 'kode' (cth: "G01") sebagai kunci.
    # Ini memastikan algoritma tidak berubah.
    ui_widgets[kode] = (checkbox, dropdown)

    # Tambahkan widget ke list untuk ditampilkan
    ui_elements_to_display.append(widgets.HBox([checkbox, dropdown]))

# Buat Tombol untuk memulai diagnosis
diagnosis_button = widgets.Button(description="📊 Mulai Diagnosis", button_style='success', layout={'width': '600px'})
# [cite_start]Buat Area Output untuk menampilkan hasil [cite: 394]
output_area = widgets.Output()

# --- 3. Fungsi Event Handler (Logika Saat Tombol Diklik) ---

# Definisikan fungsi yang akan dijalankan ketika 'diagnosis_button' diklik
def on_diagnosis_button_clicked(b):

    # 'with output_area:' memastikan semua 'print' tampil di area output
    with output_area:
        # Bersihkan output sebelumnya
        clear_output(wait=True)
        print("🔍 Menganalisis fakta...")

        # --- Kumpulkan Fakta Awal dari UI ---
        initial_facts = {}
        # Loop melalui semua widget yang kita simpan
        # 'kode' di sini akan tetap "G01", "G02", dst. karena
        # kita menyimpannya di 'ui_widgets[kode]'
        for kode, (checkbox, dropdown) in ui_widgets.items():
            # Cek apakah checkbox-nya Dicentang DAN dropdown-nya BUKAN "Pilih Keyakinan"
            if checkbox.value and dropdown.value != 0.0:
                # Jika ya, tambahkan ke 'initial_facts'
                # Format: {'G01': 0.6, 'G03': 0.6}
                initial_facts[kode] = dropdown.value

        if not initial_facts:
            print("❌ Tidak ada gejala yang dipilih atau tidak ada keyakinan yang diberikan.")
            return # Hentikan fungsi

        print(f"Fakta Awal Diterima: {initial_facts}")

        # Muat aturan dari file .json
        rules = load_rules('/content/drive/MyDrive/sispak/rules.json')

        # --- Jalankan Mesin Inferensi ---
        # Kirim fakta awal dan aturan ke mesin inferensi
        final_facts = run_inference_engine(initial_facts, rules)

        # Pisahkan hasil diagnosis (Penyakit/Komplikasi) dari fakta awal (Gejala)
        hasil_diagnosis = {k: v for k, v in final_facts.items() if k not in initial_facts}

        if not hasil_diagnosis:
            print("🏁 Tidak ada penyakit yang dapat disimpulkan dari gejala ini.")
            return

        # Urutkan hasil dari CF tertinggi ke terendah
        sorted_hasil = sorted(hasil_diagnosis.items(), key=lambda item: item[1], reverse=True)

        print("\n--- 📋 HASIL DIAGNOSIS ---")
        # Loop melalui hasil yang sudah diurutkan
        for kode, cf in sorted_hasil:
            # Ambil nama dari dictionary 'daftar_penyakit'
            nama_penyakit = daftar_penyakit.get(kode, f"Kode Tidak Dikenal ({kode})")
            # Ubah nilai CF (cth: 0.7870) menjadi persentase
            persentase = cf * 100
            # [cite_start]Tampilkan hasil (memenuhi instruksi [cite: 394])
            print(f"{nama_penyakit}: {persentase:.2f}%")

# Pasang fungsi 'on_diagnosis_button_clicked' ke tombol 'diagnosis_button'
diagnosis_button.on_click(on_diagnosis_button_clicked)

# --- 4. Tampilkan Semua Widget ---
# 'display()' adalah perintah untuk menampilkan widget ke layar Colab
# Tanda '*' (asterisk) "membongkar" list
display(*ui_elements_to_display, diagnosis_button, output_area)

--- 🩺 Sistem Pakar Deteksi Penyakit Kelinci ---
Silakan pilih gejala yang dialami kelinci dan tingkat keyakinan Anda:


Button(button_style='success', description='📊 Mulai Diagnosis', layout=Layout(width='600px'), style=ButtonStyl…

Output()